# GRADIENT BOOST MACHINE LEARNING MODEL.

Gradient boost algorithm is a type of ensemble algorithm that gradually learns from waeker learners i.e. decision tree and subsequently rectifies to improve prediction. In each training step, a learner is strengthed by minimising the loss function (MSE or class entropy) in each succeeding step in a process refered to as gradient descent. A loss function calculates the differene between actual values and predicted values, internally the model uses this value to guide output of the next model such that error is mininised as much as possible. At each iteration, the model computes the gradient of loss function for the current ensemble algorithm with respect to the previous one, learns and  trains a new weaker model to reduce gradient. This process is repeated many times, sometimes upto 1000, till a stoping criterion is met. All  weaker learners are combined to form a stronger model with potentially higher performance.  

This machine learning model is commonly due to the following reasons : 

* Higher accuracy as compared to other supervised modelling models.
* Wide ranging application, it can be used in classification and regression tasks.
* Easy to understand due to its simplicity. 

#### Algorithm Process. 

Given a dataset with x and y features, lets predict voter turnout for the year 2020 given predictor variablea voter turn out in 2016 and median age of voters. 

<center><img src="../images/GradBost1.png" width=600px height=300px /></center>

* **Step 1. Make initial Prediction.**

The first prediction is the average value of target variable , (0.63 + 0.56 + 0.8 + 0.51) / 4 = 0.625. The mean ensure the gradient of loss functions is te smallest, inthis case its 0.  

* **Step 2. Calculate residuals.** The difference between observed values and predicted values are called pseudo residuals. Lets place these values in a new column called R_1. 

<center><img src="../images/Gradbost2.png" width=700px height=300px /></center>

* **Step 3. Build a weaker learner.** 

We then build a decision tree that predicts the residials above using two features, age and 2016 voter turn out. The tree will have only 4 leave nodes. 

<center><img src="../images/GradBost3.png" width=700px height=400px /></center>

Now the data has been fitted to the output and has learnt and gained knowledge. We predict voter turnout by observing the features, when observing attributes of the first row the model, it falls on the leaf node with 0.005. 

Path followed by row 1;  

<center><img src="../images/GradBost6.png" width=700px height=300px /></center>

Thus predicted value will be, Pred_1 = 0.625(initial predicted) + (0.005) = 0.63. Consequently all values predicted will be 100% accurate when compared to actual values, Voter_turnout2020.  

<center><img src="../images/GradBost4.png" width=700px height=300px /></center>

Our model perfectly predicted voter turnout for the year 2020, but lets not be fooled, this model is overfiting and when exposed to unknown values it may cause an error. We only need it to generalise the data, this is achieved by introducing a gradient boosting parameter, the learning rate. This scaling factor multiplies the predicted values by a value between 0 and 1, when we choose 0.1, our initial value will be Pred_2 = 0.625(initial predicted)  + (0.005) * 0.1 = 0.6255.

<center><img src="../images/GradBoost5.png" width=700px height=300px /></center>

* **Step 5. Build a second pseudo-residual, R_2.** 

A second group of residuals are calculated by subtracting new predictions from original 2020 voter turnout predictions. Row 1 ,R_2 : 0.63 - 0.6255 = 0.0045 ,filled table below. 

<center><img src="../images/GradBost7.png" width=900px height=300px /></center>

* **Step 6. Iteration.** 

Take note of the gradual reduction in loss as more tree residuals are added. This step is repeated tens or even hundreds of times depending on user need. Each iteration is referred to as a boosting round. Gradient loss can be monitored to assess the nature of loss, if there no imrovement in loss the algorithm is terminated using early stop.   






